In [16]:
import yfinance as yf
import pandas as pd

symbols = ['SPY', '^RUT', '^DJI', '^STI', '^N225', '^IXIC', '^HSI', '^NSEI', '^STOXX50E']

for symbol in symbols:
    df = yf.download(symbol, start="2013-02-07", end="2023-02-07")
    df = df[['Adj Close']].copy()
    df.reset_index(inplace=True)
    df['date'] = df['Date'].dt.date
    df.rename(columns={'Date': 'datetime', 'Adj Close': symbol + '_adj_close'}, inplace=True)
    df.drop(columns=['datetime'], inplace=True)

    df.to_csv(symbol + ".csv", index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [49]:
# list to store the dataframes
dfs = []

# loop over the file names and read each file into a dataframe
for symbol in symbols:
    df = pd.read_csv(symbol + ".csv")
    dfs.append(df)

# merge the dataframes on the 'date' column
merged_df = dfs[0]
for i in range(1, len(dfs)):
    merged_df = pd.merge(merged_df, dfs[i], on='date')

merged_df

KeyError: 'date'

In [19]:
merged_df.to_csv("merged.csv", index=False)

In [21]:
import yfinance as yf
import pandas as pd

# Download the Nikkei 225 data
nikkei = yf.download("^N225", start="2020-01-01", end="2023-02-19")

# Download JPY/USD exchange rate data
jpy_usd_rate = yf.download("JPY=X", start="2020-01-01", end="2023-02-19")

# Join the dataframes on date index
nikkei_jpy = pd.DataFrame(nikkei['Adj Close'])
nikkei_jpy.reset_index(inplace=True)
nikkei_jpy['date'] = nikkei_jpy['Date'].dt.date
nikkei_jpy.rename(columns={'Date': 'datetime', 'Adj Close': 'AdjClose'}, inplace=True)
nikkei_jpy.drop(columns=['datetime'], inplace=True)


jpy_usd = pd.DataFrame(jpy_usd_rate['Close'])
jpy_usd.reset_index(inplace=True)
jpy_usd['date'] = jpy_usd['Date'].dt.date
jpy_usd.rename(columns={'Date': 'datetime', 'Adj Close': 'AdjClose'}, inplace=True)
jpy_usd.drop(columns=['datetime'], inplace=True)


nikkei_usd = pd.merge(nikkei_jpy, jpy_usd, how='inner', left_on = 'date', right_on = 'date')

# Multiply the columns to convert to USD
nikkei_usd['_Adj_Close'] = nikkei_usd['AdjClose'] / nikkei_usd['Close']

nikkei_usd[['date','_Adj_Close']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,date,_Adj_Close
0,2020-01-06,214.931459
1,2020-01-07,217.476164
2,2020-01-08,214.827063
3,2020-01-09,217.735043
4,2020-01-10,217.799497
...,...,...
758,2023-02-13,208.491926
759,2023-02-14,208.579372
760,2023-02-15,206.794895
761,2023-02-16,206.839574


In [5]:
def getdata(symbol):
    # Download data
    df = yf.download(symbol, start="2013-01-20", end="2023-02-07")
    
    # Download exchange rate data
    if symbol == '^N225':
        jpy_usd_rate = yf.download("JPY=X", start="2013-01-20", end="2023-02-07")
    elif symbol == '^HSI':
        jpy_usd_rate = yf.download("HKD=X", start="2013-01-20", end="2023-02-07")
    elif symbol == '^NSEI':
        jpy_usd_rate = yf.download("INR=X", start="2013-01-20", end="2023-02-07")
    elif symbol == '^STI':
        jpy_usd_rate = yf.download("SGD=X", start="2013-01-20", end="2023-02-07")
    elif symbol == '^STOXX50E':
        jpy_usd_rate = yf.download("EUR=X", start="2013-01-20", end="2023-02-07")
    else:
        df_jpy = pd.DataFrame(df['Adj Close'])
        df_jpy.reset_index(inplace=True)
        df_jpy['date'] = df_jpy['Date'].dt.date
        df_jpy.rename(columns={'Date': 'datetime', 'Adj Close': 'AdjClose'}, inplace=True)
        df_jpy.drop(columns=['datetime'], inplace=True)
        return(pd.DataFrame(df_jpy[['date', 'AdjClose']]))
    
    # Join the dataframes on date index
    df_jpy = pd.DataFrame(df['Adj Close'])
    df_jpy.reset_index(inplace=True)
    df_jpy['date'] = df_jpy['Date'].dt.date
    df_jpy.rename(columns={'Date': 'datetime', 'Adj Close': 'AdjClose'}, inplace=True)
    df_jpy.drop(columns=['datetime'], inplace=True)
    #print(df_jpy)

    jpy_usd = pd.DataFrame(jpy_usd_rate['Close'])
    jpy_usd.reset_index(inplace=True)
    jpy_usd['date'] = jpy_usd['Date'].dt.date
    jpy_usd.rename(columns={'Date': 'datetime', 'Close': 'Close'}, inplace=True)
    jpy_usd.drop(columns=['datetime'], inplace=True)
    #print(jpy_usd)


    df_usd = pd.merge(df_jpy, jpy_usd, how='inner', left_on = 'date', right_on = 'date')

    # Multiply the columns to convert to USD
    df_usd['_Adj_Close'] = df_usd['AdjClose'] / df_usd['Close']

    # Add dataframe to dictionary
    return(pd.DataFrame(df_usd[['date', '_Adj_Close']]))

def export_data_to_csv(symbol):
    # Download data
    df = getdata(symbol)
    
    # Export data to CSV file
    df.to_csv(symbol + ".csv", index=False)
    
    # Print a message to confirm export
    print(f"{symbol} data exported to {symbol}.csv")

import yfinance as yf
import pandas as pd

# Define list of symbols
symbols = ['SPY', '^RUT', '^DJI', '^STI', '^N225', '^IXIC', '^HSI', '^NSEI', '^STOXX50E']

# Create empty dictionary to store dataframes
data = pd.DataFrame()

# Download and convert data for each symbol
for symbol in symbols:
    export_data_to_csv(symbol)


[*********************100%***********************]  1 of 1 completed
SPY data exported to SPY.csv
[*********************100%***********************]  1 of 1 completed
^RUT data exported to ^RUT.csv
[*********************100%***********************]  1 of 1 completed
^DJI data exported to ^DJI.csv
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
^STI data exported to ^STI.csv
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
^N225 data exported to ^N225.csv
[*********************100%***********************]  1 of 1 completed
^IXIC data exported to ^IXIC.csv
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
^HSI data exported to ^HSI.csv
[*********************100%***********************]  1 of 1 completed
[*********************

In [13]:
# list to store the dataframes
dfs = []

# loop over the file names and read each file into a dataframe
for symbol in symbols:
    df = pd.read_csv(symbol + ".csv")
    dfs.append(df)
    
df = pd.read_csv("all_data.csv")
dfs.append(df)
# merge the dataframes on the 'date' column
merged_df = dfs[0]
for i in range(1, len(dfs)):
    merged_df = pd.merge(merged_df, dfs[i], on='date')

merged_df.columns = ['Date', 'SPY', 'RUT', 'DJI', 'STI', 'N225', 'IXIC', 'HSI', 'NSEI', 'STOXX50E'
                    , 'MID', 'NDX', 'NYA', 'OEX', 'RUA', 'RUI', 'SML', 'SP1500', 'SPX']
merged_df

C:\Users\Bowen\AppData\Local\Temp\ipykernel_13668\2202296434.py:14: FutureWarning: Passing 'suffixes' which cause duplicate columns {'AdjClose_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, dfs[i], on='date')
C:\Users\Bowen\AppData\Local\Temp\ipykernel_13668\2202296434.py:14: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_Adj_Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = pd.merge(merged_df, dfs[i], on='date')


,Date,SPY,RUT,DJI,STI,N225,IXIC,HSI,NSEI,STOXX50E,MID,NDX,NYA,OEX,RUA,RUI,SML,SP1500,SPX


In [61]:
merged_df.to_csv("merged.csv", index=False)

In [12]:
df_jpy = pd.read_csv("all_data.csv")
df_jpy['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.date
df_jpy

ValueError: time data '2022-01-01' does not match format '%m/%d/%Y' (match)